# Training a basic SAE with SAELens

This tutorial demonstrates training a simple, relatively small Sparse Autoencoder, specifically on the tiny-stories-1L-21M model. 

As the SAELens library is under active development, please open an issue if this tutorial is stale [here](https://github.com/jbloomAus/SAELens).

## Setup

In [1]:
import torch
import os
import sys
#import transformer-lens
sys.path.append("/n/netscratch/konkle_lab/Everyone/Jingxuan/DiffusionObjectRelation")
#import sae_lens
#from sae_lens import LanguageModelSAERunnerConfig, SAETrainingRunner
from sae_lens.sae_training_runner_custom import SAETrainingRunner
from sae_lens.config_custom import LanguageModelSAERunnerConfig
import pickle
import numpy as np
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print("Using device:", device)
os.environ["TOKENIZERS_PARALLELISM"] = "false"

/n/home03/fjxdaisy/.conda/envs/torch2/lib/python3.10/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: '/n/home03/fjxdaisy/.conda/envs/torch2/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSsb'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


Using device: cuda


### Examine activations

In [3]:
import os
import pickle

saveroot = "/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/PixArt/results/objrel_rndembdposemb_DiT_B_pilot/latent_store"

latent_file = "red_blue_8_pos_rndembposemb_img_latent_residual_prompt9_seed7.pkl"
#latent_file = "red_blue_8_pos_rndembposemb_repr_vector_dataset_all.pkl"
latent_path = os.path.join(saveroot, latent_file)

with open(latent_path, 'rb') as f:
    data = pickle.load(f)


In [4]:

def load_and_reshape_latent(pkl_path, time_step):
    """
    Loads a latent tensor from a .pkl file, extracts the block_11_residual_spatial_state_traj,
    selects a specific time step, and reshapes the tensor to batch x sequence length x feature dimension.

    Parameters:
        pkl_path (str): Path to the .pkl file.
        time_step (int): The specific time step to extract from the latent tensor (first dimension).

    Returns:
        reshaped_tensor (numpy.ndarray): Reshaped tensor of shape [batch_size, sequence_len, feature_dim].
    """
    try:
        # Load the .pkl file
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)

        # Extract the latent tensor
        latent = data['block_11_residual_spatial_state_traj']  # Shape: [time_steps, batch, 8, 8, feature_dim]

        # Check if the time step is valid
        if time_step >= latent.shape[0]:
            raise ValueError(f"Invalid time_step {time_step}. Maximum allowed is {latent.shape[0] - 1}.")

        # Extract the tensor for the given time step
        time_step_tensor = latent[time_step]  # Shape: [batch, 8, 8, feature_dim]

        # Reshape the tensor to [batch, sequence_len (8*8), feature_dim]
        batch_size, height, width, feature_dim = time_step_tensor.shape
        sequence_len = height * width
        reshaped_tensor = time_step_tensor.reshape(batch_size, sequence_len, feature_dim)

        return reshaped_tensor

    except FileNotFoundError:
        raise FileNotFoundError(f"The file at {pkl_path} was not found.")
    except KeyError:
        raise KeyError("The required key 'block_11_residual_spatial_state_traj' is missing in the .pkl file.")
    except Exception as e:
        raise RuntimeError(f"An error occurred while processing the file: {e}")



In [5]:
activations = load_and_reshape_latent(latent_path, 7)

In [6]:
activations.shape

torch.Size([50, 64, 768])

# Training an SAE

Now we're ready to train out SAE. We'll make a runner config, instantiate the runner and the rest is taken care of for us!

During training, you use weights and biases to check key metrics which indicate how well we are able to optimize the variables we care about.

To get a better sense of which variables to look at, you can read my (Joseph's) post [here](https://www.lesswrong.com/posts/f9EgfLSurAiqRJySD/open-source-sparse-autoencoders-for-all-residual-stream) and especially look at my weights and biases report [here](https://links-cdn.wandb.ai/wandb-public-images/links/jbloom/uue9i416.html).

A few tips:
- Feel free to reorganize your wandb dashboard to put L0, CE_Loss_score, explained variance and other key metrics in one section at the top.
- Make a [run comparer](https://docs.wandb.ai/guides/app/features/panels/run-comparer) when tuning hyperparameters.
- You can download the resulting sparse autoencoder / sparsity estimate from wandb and upload them to huggingface if you want to share your SAE with other.
    - cfg.json (training config)
    - sae_weight.safetensors (model weights)
    - sparsity.safetensors (sparsity estimate)

In [7]:
total_training_steps = 10  # probably we should do more
batch_size = 128
total_training_tokens = total_training_steps * batch_size

lr_warm_up_steps = 0
lr_decay_steps = total_training_steps // 5  # 20% of training
l1_warm_up_steps = total_training_steps // 20  # 5% of training

cfg = LanguageModelSAERunnerConfig(
    # Data Generating Function (Model + Training Distibuion)
    d_in=768,  # the width of the mlp output.
    # SAE Parameters
    architecture="standard",
    mse_loss_normalization=None,  # We won't normalize the mse loss,
    expansion_factor=16,  # the width of the SAE. Larger will result in better stats but slower training.
    b_dec_init_method="zeros",  # The geometric median can be used to initialize the decoder weights.
    apply_b_dec_to_input=False,  # We won't apply the decoder weights to the input.
    normalize_sae_decoder=False,
    scale_sparsity_penalty_by_decoder_norm=True,
    decoder_heuristic_init=True,
    init_encoder_as_decoder_transpose=True,
    normalize_activations="expected_average_only_in",
    # Training Parameters
    lr=5e-5,  # lower the better, we'll go fairly high to speed up the tutorial.
    adam_beta1=0.9,  # adam params (default, but once upon a time we experimented with these.)
    adam_beta2=0.999,
    lr_scheduler_name="constant",  # constant learning rate with warmup. Could be better schedules out there.
    lr_warm_up_steps=lr_warm_up_steps,  # this can help avoid too many dead features initially.
    lr_decay_steps=lr_decay_steps,  # this will help us avoid overfitting.
    l1_coefficient=5,  # will control how sparse the feature activations are
    l1_warm_up_steps=l1_warm_up_steps,  # this can help avoid too many dead features initially.
    lp_norm=1.0,  # the L1 penalty (and not a Lp for p < 1)
    train_batch_size_tokens=batch_size,
    # Activation Store Parameters
    n_batches_in_buffer=64,  # controls how many activations we store / shuffle.
    training_tokens=total_training_tokens,  # 100 million tokens is quite a few, but we want to see good stats. Get a coffee, come back.
    store_batch_size_prompts=16,
    # Resampling protocol
    use_ghost_grads=False,  # we don't use ghost grads anymore.
    feature_sampling_window=1000,  # this controls our reporting of feature sparsity stats
    dead_feature_window=1000,  # would effect resampling or ghost grads if we were using it.
    dead_feature_threshold=1e-4,  # would effect resampling or ghost grads if we were using it.
    # WANDB
    log_to_wandb=True,  # always use wandb unless you are just testing code.
    wandb_project="sae_lens_tutorial",
    wandb_log_frequency=30,
    eval_every_n_wandb_logs=20,
    # Misc
    device=device,
    seed=42,
    n_checkpoints=0,
    checkpoint_path="checkpoints",
    dtype="float32",
)
# look at the next cell to see some instruction for what to do while this is running.
#sparse_autoencoder = SAETrainingRunner(cfg).run()

In [ ]:
sparse_autoencoder = SAETrainingRunner(cfg).run()